In [1]:
import pandas as pd
import json
import numpy as np
import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client.aoligei
recomm_testset = db.recomm_testset
pred_rating_random = db.pred_rating_testset

In [3]:
db_recomm_testset = pd.read_csv('../pred_lists/file_to_store/all_movies_poster.csv')

In [4]:
db_recomm_testset

,movieId,primaryTitle,Directors,tmdb_id,genres,ratings_count,avg_rating,poster_path
0,0,Hollow Man,Paul Verhoeven,9383,Horror Sci-Fi Thriller,8898,2.546134,https://www.themoviedb.org/t/p/w600_and_h900_b...
1,1,Nurse Betty,Neil LaBute,10480,Comedy Crime Drama Romance Thriller,5259,3.165050,https://www.themoviedb.org/t/p/w600_and_h900_b...
2,2,Supernova,Walter Hill,10384,Adventure Sci-Fi Thriller,1137,2.328496,https://www.themoviedb.org/t/p/w600_and_h900_b...
3,3,Boiler Room,Ben Younger,14181,Crime Drama Thriller,5291,3.498866,https://www.themoviedb.org/t/p/w600_and_h900_b...
4,4,Pitch Black,David Twohy,2787,Horror Sci-Fi Thriller,10085,3.443778,https://www.themoviedb.org/t/p/w600_and_h900_b...
...,...,...,...,...,...,...,...,...
21634,21634,Die Geschichte vom Brandner Kaspar,Joseph Vilsmaier,17008,Comedy Drama,1,3.000000,https://www.themoviedb.org/t/p/w600_and_h900_b...
21635,21635,Rascals on the Road,Michael Steiner,15642,Children Comedy,1,4.000000,https://www.themoviedb.org/t/p/w600_and_h900_b...
21636,21636,Offroad,Elmar Fischer,83729,Comedy,1,4.000000,https://www.themoviedb.org/t/p/w600_and_h900_b...
21637,21637,Utopia Blues,Stefan Haupt,468252,(no genres listed),1,4.500000,https://www.themoviedb.org/t/p/w600_and_h900_b...


In [5]:
recomm_tojson = db_recomm_testset.to_json(orient="records")
json_recomm = json.loads(recomm_tojson)

In [6]:
recomm_testset.insert_many(json_recomm)

In [7]:
#remove null valued key-value pairs to save space
for key in list(db_recomm_testset.columns):
    recomm_testset.update_many(
#         {key: None},
        { '$or': [ { key: None }, { key: '' }, { key: [] } ] },
        {'$unset': { key: ''}}   
    )

In [8]:
random_high = pd.read_csv('../pred_lists/file_to_store/random80.csv')

In [9]:
random_high['rating'] = 'high'

In [10]:
random_low = pd.read_csv('../pred_lists/file_to_store/random_good_80.csv')

In [11]:
random_low['rating'] = 'low'

In [12]:
random = pd.concat([random_high, random_low]).reset_index()

In [13]:
random

,index,num_genres,primaryTitle,genres,originalTitle,isAdult,release_year_int,runtimeMinutes,Directors,Writers,...,company_name,keywords_name,top_10_cast_popularity,top_10_crew_popularity,top_10_crew_popularity_mean,top_10_cast_popularity_mean,tagline,homepage,belongs_to_collection,rating
0,0,3,Battling In My Enemies,"['Biography', 'Romance', 'Adventure']",Battling In My Enemies,1,2016.0,77.100794,"['None', 'Adisorn Trisirikasem']","['Vidya Talisa Ariestya', 'Yugabharathi']",...,KCD Films,"['vermont', 'supermarket']","[1.063, 0.6, 2.769, 4.682, 0.6, 0.6, 4.091]","[0.6, 0.717, 0.6, 0.694, 0.6, 0.6]",0.635167,2.057857,Sometimes the toughest battles are fought in y...,http://movies.disney.com/remember-the-titans,Tiger Claws Collection,high
1,1,3,Searching At The Void,"['Family', 'Reality-TV', 'Adventure']",Searching At The Void,1,2016.0,109.697752,"['Richard Trank', 'Samuel Galli']","['Petr Jarchovský', 'Ethan Coen']",...,Chozha Creations,"['demonic possession', 'female scientist']","[0.6, 2.022, 1.092, 3.346]","[0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]",0.600000,1.765000,No one stays innocent forever,None,None,high
2,2,2,Praised By My Nightmares,"['History', 'Animation']",Praised By My Nightmares,0,2005.0,97.460148,"['Helena Vsetecková', 'Rebecca Richman Cohen']","['Josef Urban', 'Scott Baehrend']",...,P.J. Entertainments,"['self-fulfilling prophecy', 'galaxy']","[0.6, 0.6, 0.6, 3.346, 2.316]","[0.6, 0.6, 1.586, 0.6, 0.6, 0.98, 0.6, 0.6, 0.6]",0.751778,1.492400,He's Never Been To Earth. He's Never Even Slep...,http://in-crowd.warnerbros.com/,"Monsters, Inc. Collection",high
3,3,4,Vanish At The Darkness,"['Crime', 'Reality-TV', 'Sci-Fi', 'Adventure']",Vanish At The Darkness,1,2021.0,86.330586,"['Andrew Jenks', 'None']","['Stephen Langford', 'Junichi Masuda']",...,FORT ENTERTAINMENT,"['old friend', 'diner']","[0.683, 1.052, 1.4, 0.6, 1.22, 1.178, 0.745, 1...","[0.6, 0.6, 0.6]",0.600000,1.089444,The only way to find out what you love is to r...,https://www.miramax.com/movie/amelie/,None,high
4,4,1,Driving Into The Ocean,['Romance'],Driving Into The Ocean,1,2012.0,98.556001,"['Tom Bean', 'None']","['Junichi Masuda', 'Lau Ho-Leung']",...,Canal 9 Televisió Valenciana,"['attempted suicide', 'literature']","[0.954, 0.6, 0.98, 1.547, 0.6, 2.336, 3.502, 0.6]",[0.694],0.694000,1.389875,This summer it's not the same old bull.,None,Qatsi Collection,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,75,1,Forsaken By My Past,['Documentary'],Forsaken By My Past,1,2007.0,89.872455,"['Tang Shi', 'Hong-jin Na', 'Ariadne Asimakopo...","['Jane Anderson', 'Jennie Marie Pacelli', 'Pau...",...,Pauline's Angel,"['drug addiction', 'police brutality']","[8.58111111111111, 3.689500000000001, 2.565222...","[2.435, 1.3999999999999997, 1.4, 1.39076470588...",2.354762,5.242524,Same Planet. New Scum.,http://www.brockovich.com/movie.html,None,low
156,76,3,Rejecting My Family,"['War', 'History', 'Documentary']",Rejecting My Family,1,2009.0,116.907025,"['James Reid', 'Ryan Caraway', 'Erik Cimon']","['Howard Franklin', 'Billy Kimball', 'Steve Ar...",...,C4 Productions,"['computer animation', 'lucifer']","[3.748, 2.6636, 2.744, 2.369, 3.576, 5.5818, 7...","[1.4, 1.4, 1.3542857142857136, 1.6100000000000...",1.579611,4.015363,“I enjoyed it and found it very interesting an...,http://www.lagaan.com,Hollow Man Collection,low
157,77,4,Binding To The Hunter,"['History', 'Music', 'War', 'Documentary']",Binding To The Hunter,1,2004.0,106.002034,"['Kate McLean', 'R. Scott Leisk', 'Sean Patric...","['Tom Nursall', 'Ercan Mehmet Erdem', 'Ken Ran...",...,Máquina de Películas,"['keks', 'invisible person']","[6.825666666666667, 3.294, 3.155333333333333, ...","[13.254, 1.4, 1.708, 1.4, 1.5455, 1.6489999999...",3.531026,4.935095,Sex and Drugs before Rock and Roll,http://www.ballardian.com/weiss-atrocity-exhib...,MVP,low
158,78,3,Smiles In The Forest,"['War', 'Documentary', 'History']",Smiles In The Forest,0,2009.0,93.713702,"['Phillip Galloway'

In [14]:
random_tojson = random.to_json(orient="records")
json_random = json.loads(random_tojson)

In [15]:
pred_rating_random.insert_many(json_random)

In [16]:
#remove null valued key-value pairs to save space
for key in list(random.columns):
    pred_rating_random.update_many(
#         {key: None},
        { '$or': [ { key: None }, { key: '' }, { key: [] } ] },
        {'$unset': { key: ''}}   
    )

In [17]:
import glob
csvs = glob.glob("../pred_lists/file_to_store/lists/*.csv")

In [18]:
to_insert = {'pred_rating_lists': 1}
for path in csvs:
    df = pd.read_csv(path)
    first_col_name = df.columns[0]
    if path.split('/')[-1].startswith('top'):
        first_col_name = 'high_' + first_col_name
    elif path.split('/')[-1].startswith('low'):
        first_col_name = 'low_' + first_col_name
    first_col = df.iloc[:,0].values.tolist()
    to_insert[first_col_name] = {first_col_name: first_col}
    
pred_rating_random.insert_one(to_insert)

In [44]:
dd = pred_rating_random.find({'pred_rating_lists': 1}, { 'high_casts': 1})[0]

In [81]:
k k= list(dd['high_casts'].values())[0]

SyntaxError: invalid syntax (<ipython-input-81-98f25b8681c0>, line 1)

In [59]:
dd['_ifd']

KeyError: '_ifd'

In [77]:
cursor = recomm_testset.aggregate([
    { '$project': {'_id': 0 } },
    { '$match': { 'poster_path': { '$exists': 1 } } }
])

In [78]:
h = list(cursor)

In [76]:
len(h)

21639

In [79]:
len(h)

20435

In [82]:
k

['Aletta Ocean',
 'Johnny Depp',
 'Scarlett Johansson',
 'Millie Bobby Brown',
 'Gal Gadot',
 'Jason Statham',
 'Rosa Caracciolo',
 'Henry Cavill',
 'Bunny Colby',
 'Jayden Jaymes',
 'Zack Snyder',
 'Holly Halston',
 'Margot Robbie',
 'Leonardo DiCaprio',
 'Dwayne Johnson',
 'Chloë Grace Moretz',
 'Nicolas Cage',
 'Tom Hardy',
 'Bruce Willis',
 'Kayden Kross',
 'Anne Hathaway',
 'Taegen Burns',
 'Cathy Heaven',
 'Alexandra Daddario',
 'Tom Cruise',
 'Milla Jovovich',
 'Shyla Stylez',
 'Ben Affleck',
 'Anya Taylor-Joy',
 'Anthony Hopkins',
 'Robert Downey Jr.',
 'Marica Hase',
 'Tori Black',
 'Evangeline Lilly',
 'Arnold Schwarzenegger',
 'Sebastian Stan',
 'Liam Neeson',
 'Connie Nielsen',
 'Elizabeth Olsen',
 'Sean Bean',
 'Rosamund Pike',
 'Eiza González',
 'Bridgette B',
 'Cherie DeVille',
 'Dee Williams',
 'Jet Li',
 'Vin Diesel',
 'Will Smith',
 'Emily Willis',
 'Ryan Reynolds',
 'Danielle Panabaker',
 'Sylvester Stallone',
 'Pedro Pascal',
 'Denzel Washington',
 'Natalie Portman'